# Deconvolution with Besov Priors

This notebook demonstrates the use of Besov priors for solving deconvolution problems using Bayesian inference.
The workflow includes defining the test signal, setting up the forward model, and performing posterior sampling using RTO-MH and NUTS samplers

In [ ]:
# Import required libraries and modules
from blurring import blurring
from besov_prior import besov_prior
from inverse_problem import inverse_problem
from rto_mh import rto_mh
import numpy as np
import cuqi
import time
from scipy.special import gamma
import arviz as az

## Step 1: Define the Test Signal
We define a piecewise test signal with multiple intervals to simulate the ground truth

In [ ]:
def Test_Signal(x):
    """
    Generate a piecewise smooth test signal.
    Args:
        x (np.ndarray): Input array of spatial points.
    Returns:
        Signal (np.ndarray): Piecewise smooth test signal.
    """
    Signal = np.zeros(len(x))
    interval_1 = np.logical_and(0.1 <= x, x <= 0.35)
    interval_2 = np.logical_and(0.35 < x, x <= 0.45)
    interval_3 = np.logical_and(0.45 < x, x <= 0.60)
    interval_4 = np.logical_and(0.60 < x, x < 0.90)
    # Define signal values for each interval
    Signal[interval_1] = np.exp(-(x[interval_1]-0.35)**2*150)
    Signal[interval_2] = 1.0
    Signal[interval_3] = 0.8
    Signal[interval_4] = 0.40
    return Signal

## Step 2: Set Up the Forward Model
We set up the forward model using the `blurring` class, which applies a Gaussian blur to the input signal.

In [ ]:
# Initialize the test signal and forward model
np.random.seed(5) # Set random seed for reproducibility
N = 4096 # Number of spatial points
x = np.linspace(0, 1, N, endpoint=False) # Spatial grid
signal = Test_Signal(x) # Generate the test signal

# Set up the blurring forward model
likelihood = blurring(x,sigma_kernel=0.02)  # Gaussian kernel with sigma=0.02
likelihood.set_data(signal,noise_level=2.0) # Add Gaussian noise with relative noise level of 2.0

# Save relevant data for later use
lam = likelihood.lam # Noise precision
Data = likelihood.data[0::128] # Subsampled data
m = 32 # Number of observed data points

## Step 3: Bayesian Inference with different unknown dimension
We perform Bayesian inference using the Randomize-Then-Optimize (RTO) sampler with varying wavelets and unknown dimension

In [ ]:
np.random.seed(5)
n_range = [32, 64, 128, 256, 512, 1024, 2048] # Different spatial resolutions
delt = 1 # Regularization parameter
wavelets = ['db1','db8']  # Wavelet types
level = 0 
nsamp = 10000 # Number of samples

# Loop through wavelet types and spatial resolutions
for wavelet in wavelets:
    J = 5 # Initial wavelet level
    for n in n_range:
            x_n = np.linspace(0, 1, n, endpoint=False)
            x0 = np.ones(len(x_n)) # Initial guess for optimization

            # Set up the forward model and prior
            likelihood = blurring(x_n,sigma_kernel=0.02, Factor= int(n/m))
            likelihood.data = Data
            likelihood.lam = lam
            prior = besov_prior(J=J, delt=delt, level=level,s=1.0,p=1.5,wavelet=wavelet)
            jac_const = likelihood.jac_const(n) @ prior.jac_const(n)

            # Define the inverse problem
            problem = inverse_problem(likelihood,prior,jac_const)
            Nrand = m+n

             # Perform RTO sampling
            rto_sampler=rto_mh(x0,Nrand,samp=nsamp)
            z_Map=rto_sampler.initialize_Q(problem)
            rto_sampler.x0 = z_Map
            xchain, acc_rate, index_accept, log_c_chain = rto_sampler.sample(problem)

            # Save the results
            np.save("Discrete_Invariant_Samples_"+wavelet + "_n=" + str(n) + "_s=1.0_p=1.5" + ".npz",xchain)
            np.save("Discrete_Invariant_index_accept_"+wavelet+"_n=" + str(n)+ "_s=1.0_p=1.5" +  ".npz",index_accept)
            print(acc_rate) # Print acceptance rate
            J += 1 # Increment wavelet level

##  Comparison between RTO and NUTS sampling

We perform Bayesian inference using the Randomize-Then-Optimize (RTO) sampler and compare it with the No-U-Turn Sampler (NUTS).

In [ ]:
np.random.seed(5) # Set random seed for reproducibility
# Set up the test signal and forward model
J = 9 # Wavelet level
x=np.linspace(0, 1, 2**J, endpoint=False) # Spatial grid
signal = Test_Signal(x) # Test signal

# Setting up the forward model and generating the data
likelihood = blurring(x,sigma_kernel=0.02)
likelihood.set_data(signal,noise_level=2.0)

# Save data for later use
n = len(x) # Dimension of the problem
m = len(likelihood.data) # Dimension of the data

In [ ]:
## Setting up the prior, inverse problem, and RTO sampler

# Initializing prior parameters
p = 1.5
s = 1.0
wavelet = 'db1'
level = 0
delt = 1.0
prior = besov_prior(J = J,delt=delt, level=level,s=s,p=p,wavelet=wavelet)

# Setting up the inverse problem
jac_const = likelihood.jac_const(n) @ prior.jac_const(n)
problem = inverse_problem(likelihood,prior,jac_const)

# Setting up the RTO sampler
Nrand = n+m
x0 = np.ones(n)
nsamp = 1400
rto_sampler=rto_mh(x0,Nrand,samp=nsamp)

# Initializing the RTO sampler
zMap=rto_sampler.initialize_Q(problem)
MAP = prior.transform(zMap)

NUTS sampling using CUQIpy

In [ ]:
np.random.seed(5) # Random seed for reproducibility

# Defining the log-posterior and gradient functions of the prior
logpdf = lambda x:-delt/((np.sqrt(gamma(1/p)/gamma(3/p)))**p)*np.linalg.norm(prior.wavelet_weigth(x),ord=p)**p
gradient = lambda x:-p*delt/((np.sqrt(gamma(1/p)/gamma(3/p)))**p)*prior.wavelet_weight_adjoint(np.sign(prior.wavelet_weigth(x))*np.abs(prior.wavelet_weigth(x))**(p-1))

# Putting the logpdf and gradient together defining the prior distribution
xx = cuqi.distribution.UserDefinedDistribution(dim=n,logpdf_func=logpdf,gradient_func=gradient)

# Setting up the forward model
model = cuqi.model.LinearModel(likelihood.jac_const(n))

# Defining the likelihood distribution
y = cuqi.distribution.Gaussian(model(xx),likelihood.lam**2)

# Setting up the joint distribution
joint = cuqi.distribution.JointDistribution(y,xx)

# Conditioning on the data to obtain the posterior distribution
posterior = joint(y=likelihood.data)

# Setting up the NUTS sampler
sampler = cuqi.sampler.NUTS(posterior,MAP,adapt_step_size=True,opt_acc_rate=0.8)

# Sampling from the posteiror distribution
t0 = time.process_time()
chain_NUTS = sampler.sample(1000,400)
t1 = time.process_time()

# Calculat the total time taken for sampling
total_time_NUTS = t1-t0
print(total_time_NUTS)

# Saving results for further analysis
np.save("Comparison_samples_deconvolution_NUTS.npy",chain_NUTS.samples)
np.save("Time_NUTS.npy",total_time_NUTS)


Sample 1400 / 1400
634.859375


RTO sampling

In [ ]:
np.random.seed(5) # random seed for reproducibility

# Initializing the start point of the RTO sampler
rto_sampler.x0=zMap

# Computing the samples
t0 = time.process_time()
chain_RTO, acc_rate, index_accept ,log_c_chain  = rto_sampler.sample(problem)
t1 = time.process_time()

# Calculating the total time taken for sampling
total_time_RTO = t1-t0

# Printing results
print(acc_rate)
print(total_time_RTO)
chain_RTO_accept = chain_RTO[:,index_accept]

# Saving results for further analysis
np.save("Comparison_samples_deconvolution_RTO.npy",chain_RTO_accept[:,0:1000])
np.save("Time_RTO.npy",total_time_RTO)

0.7057142857142857
757.59375


Effeciency comparison

In [ ]:
# Loading results RTO results and computing the effective sample size (ESS)
chain_RTO = np.load("Comparison_samples_deconvolution_RTO.npy")
time_RTO = np.load("Time_RTO.npy")
RTO_samples = chain_RTO
ESS_RTO = np.zeros(n)
for i in range(n):
    ESS_RTO[i]=az.ess(RTO_samples[i,:])

# Print ESS and ESS/s    
print(ESS_RTO.min(),np.median(ESS_RTO),ESS_RTO.max())
print(ESS_RTO.min()/time_RTO,np.median(ESS_RTO)/time_RTO,ESS_RTO.max()/time_RTO)

# Loading NUTS results and computing ESS
chain_NUTS=np.load("Comparison_samples_deconvolution_NUTS.npy")
time_NUTS = np.load("Time_NUTS.npy")
ESS_NUTS = np.zeros(n)
for i in range(n):
    ESS_NUTS[i]=az.ess(chain_NUTS[i,:])

# Print ESS and ESS/s    
print(ESS_NUTS.min(),np.median(ESS_NUTS),ESS_NUTS.max())
print(ESS_NUTS.min()/time_NUTS,np.median(ESS_NUTS)/time_NUTS,ESS_NUTS.max()/time_NUTS)

580.653631570516 954.1456042915211 1178.2184998559271
0.7664445906140541 1.2594422859105174 1.5552114835370898
240.20501208935383 677.5247660702041 1064.8625096460817
0.3783593998109484 1.0672044751173504 1.6773202878922306
